## Note: things to do to improve before pushing to github:
* similar to mini-project-I, remove cells with collapse json files, lists and what not. Makes the notebook too hard to view 

In [36]:
import requests
import os
import json
from IPython.display import JSON
import numpy as np
import pandas as pd

In [37]:
import sqlite3
from sqlite3 import Error

# Yelp API

In [5]:
from yelpapi import YelpAPI

In [4]:
yelp_API_key = os.environ["YELP_API_KEY"]

In [14]:
base_url = "https://api.yelp.com"
# will most likely be using following urls (host+path):
# https://api.yelp.com/v3/businesses/search
# https://api.yelp.com/v3/businesses/{id}/reviews

In [41]:
HOST = 'https://api.yelp.com'
path1 = '/v3/business/search'
headers = {"Authorization" : f"Bearer {yelp_API_key}"}
url_params1 = {'term': 'bar', 'location': 'Vancouver, BC'}

In [5]:
url_params2 = {}

In [6]:
url_params3 = {}

In [42]:
response = requests.get("https://api.yelp.com/v3/businesses/search", headers=headers, params=url_params1)

In [43]:
response.status_code

200

In [46]:
json_yelp = response.json()

In [175]:
# data = json.load(open('json_file.json'))

# df = pd.DataFrame(data["result"])
# possible to create dataframe directly from the json results?

In [48]:
pretty_json_yelp = JSON(json_yelp)

In [2]:
# pretty_json_yelp

In [116]:
# retrieving data
for key, value in json_yelp.items():
    if json_yelp[key] == json_yelp['businesses']:
        restaurant_list = list(value)

In [180]:
# getting names
list_names = []

for restaurant in restaurant_list:
    for key, value in restaurant.items():
        if restaurant[key] == restaurant['name']:
            list_names.append(value)

In [181]:
# getting price ranges
list_price_range = []

for restaurant in restaurant_list:
    try:
        for key, value in restaurant.items():
            if restaurant[key] == restaurant['price']:
                list_price_range.append(value)
    except:
        list_price_range.append(np.nan)

In [182]:
# getting rating
list_rating = []

for restaurant in restaurant_list:
    try:
        for key, value in restaurant.items():
            if restaurant[key] == restaurant['rating']:
                list_rating.append(float(value))
    except:
        list_rating.append(np.nan)

In [183]:
# getting review count
list_review = []

for restaurant in restaurant_list:
    try:
        for key, value in restaurant.items():
            if restaurant[key] == restaurant['review_count']:
                list_review.append(int(value))
    except:
        list_review.append(np.nan)

In [184]:
# getting category
list_category1 = []

for restaurant in restaurant_list:
    try:
        for key, value in restaurant.items():
            if restaurant[key] == restaurant['categories']:
                list_category1.append(restaurant[key][0]['title'])
    except:
        list_category1.append(np.nan)

In [185]:
list_category2 = []

for restaurant in restaurant_list:
    try:
        for key, value in restaurant.items():
            if restaurant[key] == restaurant['categories']:
                list_category2.append(restaurant[key][1]['title'])
    except:
        list_category2.append(np.nan)

In [186]:
# creating dataframe
dict_yelp = {}
dict_yelp['name'] = list_names
dict_yelp['price_range'] = list_price_range
dict_yelp['rating'] = list_rating
dict_yelp['review_count'] = list_review
dict_yelp['category_1'] = list_category1
dict_yelp['category_2'] = list_category2

In [188]:
yelp_data = pd.DataFrame(dict_yelp)

In [189]:
yelp_data.head()

,name,price_range,rating,review_count,category_1,category_2
0,Prohibition,$$,4.0,144,Lounges,Cocktail Bars
1,The Shameful Tiki Room,$$,4.0,304,Tiki Bars,NaN
2,Alibi Room,$$,4.5,535,Pubs,Canadian (New)
3,The Keefer Bar,$$,4.0,239,Lounges,NaN
4,Key Party,NaN,4.5,11,Lounges,NaN


In [59]:
# creating sqlite file
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [60]:
connection = create_connection("yelp_results.sqlite")

Connection to SQLite DB successful


In [58]:
# for getting results once the database is complete
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [194]:
# dataframe to sql
yelp_data.to_sql(name='yelp_results_table', con=connection)

In [197]:
# how to make queries to sql file
# get_all = pd.read_sql('SELECT * From yelp_results_table', connection)

### Some results

In [217]:
restaurant_prices = pd.read_sql("""
SELECT price_range, COUNT(name)
FROM yelp_results_table
GROUP BY price_range
ORDER BY COUNT(name) DESC
""", connection)

restaurant_prices.head()

,price_range,COUNT(name)
0,$$,14
1,None,3
2,$,2
3,$$$,1


In [220]:
most_frequent_cat = pd.read_sql("""
SELECT category_1, COUNT(category_1) 
FROM yelp_results_table
GROUP BY category_1
ORDER BY COUNT(category_1) DESC
""", connection)

In [221]:
most_frequent_cat

,category_1,COUNT(category_1)
0,Lounges,5
1,Tiki Bars,3
2,Pubs,3
3,Cocktail Bars,3
4,Dive Bars,2
5,Wine Bars,1
6,Pizza,1
7,Music Venues,1
8,Bars,1


In [222]:
most_reviews = pd.read_sql("""
SELECT name, MAX(review_count)
FROM yelp_results_table
""", connection)

most_reviews

,name,MAX(review_count)
0,Alibi Room,535


### Top 10 restaurant based on rating

In [216]:
most_pop = pd.read_sql("""
SELECT name, rating
FROM yelp_results_table
ORDER BY rating DESC
""", connection)

most_pop.head(10)

,name,rating
0,The Red Accordion,5.0
1,Alibi Room,4.5
2,Key Party,4.5
3,Jackalope's Neighbourhood Dive,4.5
4,Grapes & Soda,4.5
5,1927 Lobby Lounge,4.5
6,Prohibition,4.0
7,The Shameful Tiki Room,4.0
8,The Keefer Bar,4.0
9,The Tiki Bar,4.0


# Foursquare API

In [3]:
foursquare_client_id = os.environ["FOURSQUARE_CLIENT_ID"]
foursquare_client_secret = os.environ["FOURSQUARE_CLIENT_SECRET"]

In [ ]:
# 49.2827° N, 123.1207° W cords vancouver

In [4]:
response_foursquare = response = requests.get(f"https://api.foursquare.com/v2/venues/explore?client_id={foursquare_client_id}&client_secret={foursquare_client_secret}&ll=49.2827,-123.1207&v=20200731")

In [5]:
response_foursquare.status_code

200

In [14]:
json_foursquare = response_foursquare.json()

In [11]:
pretty_foursquare_json = JSON(json_foursquare)

In [1]:
# pretty_foursquare_json

In [29]:
# retrieving some data:
for key in json_foursquare:
    if json_foursquare[key] == json_foursquare["response"]:
        response = json_foursquare[key]
        for key in response:
            if response[key] == response["groups"]:
                groups = response[key]
                for key, value in groups[0].items():
                    if groups[0][key] == groups[0]["items"]:
                         list_items = list(value)


In [44]:
clearer_list = JSON(list_items)

In [42]:
# retrieving name of locations
list_venue_names = []

for item in list_items:
    for key in item:
        if item[key] == item["venue"]:
            venue = item[key]
            for key, values in venue.items():
                if venue[key] == venue["name"]:
                    list_venue_names.append(values)
len(list_venue_names)

30

In [43]:
list_ratings = []

for item in list_items:
    for key in item:
        if item[key] == item["reasons"]:
            reasons = item[key]
            for key in reasons:
                if reasons[key] == reasons["items"]:
                    list_ratings.append(reasons[key][0]['summary'])

len(list_ratings)

30

In [3]:
# clearer_list

In [120]:
list_category = []

for item in list_items:
    for key in item:
        if item[key] == item["venue"]:
            venue = item[key]
            for key, value in venue.items():
                if venue[key] == venue["categories"]:
                    categories = venue[key]
                    for things in categories:
                        list_category.append(things)

In [119]:
final_list_category = []

for items in list_category:
    for key, value in items.items():
        if items[key] == items['pluralName']:
            final_list_category.append(value)

len(final_list_category)

30

In [122]:
dict_foursquare = {}
dict_foursquare['name'] = list_venue_names
dict_foursquare['rating'] = list_ratings
dict_foursquare['category'] = final_list_category

In [123]:
foursquare_data = pd.DataFrame(dict_foursquare)

In [125]:
foursquare_data.head(30)

,name,rating,category
0,Rosewood Hotel Georgia,This spot is popular,Hotels
1,Hawksworth Restaurant,This spot is popular,Lounges
2,Disney store,This spot is popular,Toy / Game Stores
3,Victoria's Secret,This spot is popular,Lingerie Stores
4,Abercrombie & Fitch,This spot is popular,Clothing Stores
5,Wedgewood Hotel,This spot is popular,Hotels
6,Commodore Ballroom,This spot is popular,Concert Halls
7,Vancouver Art Gallery,This spot is popular,Art Galleries
8,Rocky Mountain Chocolate Factory,This spot is popular,Dessert Shops
9,Hubbub Sandwiches,This spot is popular,Sandwich Places
